In [106]:
class Link:
    data = None

    @classmethod
    def get(cls):
        return cls.data

    @classmethod
    def set(cls, data):
        cls.data = data


class MissingDataLink(Link):
    pass


class PrepLink(Link):
    pass


class InitLink(Link):
    pass


def md_link(func):
    def inner(self, *args, **kwargs):
        func(self, *args, **kwargs)
        data = self.data
        MissingDataLink.set(data)

    return inner


In [107]:
class BaseGetSet:
    def __init__(self, data):
        self.data = data

    @property
    def data(self):
        return self._data

    @data.setter
    def data(self, data):
        Link.set(data)
        self._data = Link.get()

    @data.getter
    def data(self):
        self._data = Link.get()
        return Link.get()


class Core(BaseGetSet):
    def __init__(self, data: int):
        super().__init__(data)
        InitLink.set(data)
        self.missing_data = MissingData(Link.get())
        self.preprocessing = Preprocessing(Link.get())


class MissingData(BaseGetSet):
    def __init__(self, data):
        super().__init__(data)

    @md_link
    def md_stuff(self, number):
        self.data = self.data + number

    def reset(self):
        self.data = InitLink.get()


class Preprocessing(BaseGetSet):
    def __init__(self, data):
        super().__init__(data)
        PrepLink.set(data)

    def prep_stuff(self, number):
        self.data = self.data * number

    def reset(self):
        self.data = MissingDataLink.get()


core = Core(2)

In [108]:
core.missing_data.data

2

In [109]:
core.missing_data.md_stuff(3)


In [110]:
core.data

5

In [111]:
core.missing_data.reset()

In [112]:
core.missing_data.data

2

In [113]:
core.missing_data.md_stuff(3)

In [114]:
core.data

5

In [118]:
core.preprocessing.prep_stuff(2)

In [116]:
core.data

10

In [117]:
core.preprocessing.reset()
core.data

5

In [119]:
core.data

10

In [120]:
core.missing_data.reset()

In [121]:
core.data

2

In [153]:
from dataclasses import dataclass


@dataclass
class LinkInst:
    data: [int | None]

    def get(self):
        return self.data

    def set(self, data):
        self.data = data


In [154]:
link1 = LinkInst(None)
link2 = LinkInst(None)
link3 = LinkInst(None)


class BaseGetSet:
    def __init__(self, data):
        self.data = data

    @property
    def data(self):
        return self._data

    @data.setter
    def data(self, data):
        link1.set(data)
        self._data = link1.get()

    @data.getter
    def data(self):
        self._data = link1.get()
        return link1.get()


class Core(BaseGetSet):
    def __init__(self, data: int):
        super().__init__(data)
        link2.set(data)
        self.missing_data = MissingData(link1.get())
        self.preprocessing = Preprocessing(link1.get())


class MissingData(BaseGetSet):
    def __init__(self, data):
        super().__init__(data)

    @md_link
    def md_stuff(self, number):
        self.data = self.data + number

    def reset(self):
        self.data = link1.get()


class Preprocessing(BaseGetSet):
    def __init__(self, data):
        super().__init__(data)
        link2.set(data)

    def prep_stuff(self, number):
        self.data = self.data * number

    def reset(self):
        self.data = link3.get()


core = Core(2)

In [155]:
core.preprocessing.data

2

In [156]:
core.data = 1

In [159]:
core.missing_data.data

1

In [161]:
core.missing_data.md_stuff(2)
core.data

3

In [164]:
core.preprocessing.prep_stuff(3)
core.data

27

In [165]:
core.preprocessing.reset()

2